In [59]:
import sys  
sys.path.insert(1, '..')
sys.path.insert(2, '../modules/')
import re
import pickle
import pandas as pd
from time import sleep
from random import randint
import time
import sys
import os
import hashlib
from ruffus import *
import sqlalchemy as sa
import warnings
warnings.filterwarnings("ignore")
import numpy as np
from glob import glob

# argonne libraries

# from download_module_argonne import *

from modules import download_module_w_new_file_syst
from modules import query_the_data
import dataconfig
from modules import convert_datetime
import helio_reg_exp_module
from glob import glob

In [60]:
def convert_int_class_to_float(flare_class):

    """ 
        make sure flare class is not C1 or M1 but C1.0 or M1.0 etc.
    """
    
    if len(flare_class) == 2:

        flare_letter = flare_class[:1]
        flare_number = np.float(flare_class[1:])
        flare_class = f'{flare_letter}{flare_number}'

    return(flare_class)


In [61]:
# {'merged_datetime': {100: Timestamp('2011-02-08 21:11:00.081666816+0000', tz='UTC')},
#  'id_tuple': {100: (1, 1, 1)},
#  'merged_value': {100: 4e-06},
#  'merged_class': {100: 'C4.0'}}

# flarecandidate_C1.8_at_2011-11-14T20_12_30_25.working

In [91]:
# user input time in the following format 'YYYY-MM-DD HH:MM:SS.ff' in UTC
input_time_list = ['2011-02-08 21:11:00.08', '2011-11-14 20:12:30.25']

this_time_list = [pd.Timestamp(input_time, tz='UTC') for input_time in input_time_list]

this_time_list

[Timestamp('2011-02-08 21:11:00.080000+0000', tz='UTC'),
 Timestamp('2011-11-14 20:12:30.250000+0000', tz='UTC')]

In [92]:
input_class_list = ['C4.0', 'C1.8']

In [110]:
def convert_int_class_to_float(flare_class):

    """ 
        make sure flare class is not C1 or M1 but C1.0 or M1.0 etc.
    """
    
    if len(flare_class) == 2:
        flare_letter = flare_class[:1]
        flare_number = np.float(flare_class[1:])
        flare_class = f'{flare_letter}{flare_number}'

    return(flare_class)

def create_working_dir(flare_class, flare_time):
    this_file_time = f'{flare_time:%Y-%m-%dT%H_%M_%S_%f}'[:-4] # cut the milisecond precision to seconds
    name_of_directory = f'flarecandidate_{flare_class}_at_{this_file_time}.working'
    return(name_of_directory)

tw = lambda x: os.path.join(WORKING_DIR, x)

## end of some functions

# workdir variable
WORKING_DIR = dataconfig.DATA_DIR_FLARE_CANDIDATES

In [111]:
THESE_FLARES_ARGONNE_dict = [{'merged_datetime':this_time,'merged_class': this_class, 'working_dir':  tw(create_working_dir(this_class, this_time))} for this_time, this_class in zip(this_time_list, input_class_list)]
THESE_FLARES_ARGONNE_dict

[{'merged_datetime': Timestamp('2011-02-08 21:11:00.080000+0000', tz='UTC'),
  'merged_class': 'C4.0',
  'working_dir': '/mnt/e/ALEXIS_data/flare_candidates/flarecandidate_C4.0_at_2011-02-08T21_11_00_08.working'},
 {'merged_datetime': Timestamp('2011-11-14 20:12:30.250000+0000', tz='UTC'),
  'merged_class': 'C1.8',
  'working_dir': '/mnt/e/ALEXIS_data/flare_candidates/flarecandidate_C1.8_at_2011-11-14T20_12_30_25.working'}]

In [134]:
THESE_NEW_FLARES = pd.DataFrame(THESE_FLARES_ARGONNE_dict)

THESE_NEW_FLARES

,merged_datetime,merged_class,working_dir
0,2011-02-08 21:11:00.080000+00:00,C4.0,/mnt/e/ALEXIS_data/flare_candidates/flarecandi...
1,2011-11-14 20:12:30.250000+00:00,C1.8,/mnt/e/ALEXIS_data/flare_candidates/flarecandi...


In [135]:
flare_candidate_dir = dataconfig.DATA_DIR_FLARE_CANDIDATES
WD_already_done = glob(f'{flare_candidate_dir}/*.working')

In [136]:
this_wd_list = ([helio_reg_exp_module.work_dir_from_flare_candidate_input_string(this_wd_path) for this_wd_path in WD_already_done])

this_merged_datetime = [helio_reg_exp_module.date_time_from_flare_candidate_working_dir(this_wd) for this_wd in this_wd_list]

this_class = [helio_reg_exp_module.flare_class_from_flare_candidate_working_dir(this_wd) for this_wd in this_wd_list]

ouput_dict_list = [{'merged_datetime': this_datetime, 'merged_class': this_class, 'working_dir': this_wd} for this_datetime, this_class, this_wd in zip(this_merged_datetime, this_class, WD_already_done )]

WD_ALREADY_DONE = pd.DataFrame(ouput_dict_list)



In [140]:
WD_ALREADY_DONE

,merged_datetime,merged_class,working_dir
0,2011-11-14 20:12:30+00:00,C1.8,/mnt/e/ALEXIS_data/flare_candidates/flarecandi...
1,2011-02-08 21:11:00+00:00,C4.0,/mnt/e/ALEXIS_data/flare_candidates/flarecandi...


In [141]:
THESE_NEW_FLARES

,merged_datetime,merged_class,working_dir
0,2011-02-08 21:11:00.080000+00:00,C4.0,/mnt/e/ALEXIS_data/flare_candidates/flarecandi...
1,2011-11-14 20:12:30.250000+00:00,C1.8,/mnt/e/ALEXIS_data/flare_candidates/flarecandi...


In [139]:
os.path.exists(wd_to_do[0]),os.path.exists(wd_to_do[1])

(True, True)

In [117]:
def make_wd_if_wd_not_there(directory):

    if not os.path.exists(directory):
        # Create the directory
        os.makedirs(directory)
        # os.makedirs(self.MAIN_DL_DIRECTORY)
        print(f'Directory "{directory}" created.')
        return(directory)
    else:
        pass

wd_made = [make_wd_if_wd_not_there(this_wd) for this_wd in THESE_NEW_FLARES.working_dir]

In [143]:
wd_made = [make_wd_if_wd_not_there(this_wd) for this_wd in THESE_NEW_FLARES.working_dir]


In [144]:
THESE_FLARES

,merged_datetime,merged_class,working_dir
0,2011-02-08 21:11:00.080000+00:00,C4.0,/mnt/e/ALEXIS_data/flare_candidates/flarecandi...
1,2011-11-14 20:12:30.250000+00:00,C1.8,/mnt/e/ALEXIS_data/flare_candidates/flarecandi...
0,2011-11-14 20:12:30+00:00,C1.8,/mnt/e/ALEXIS_data/flare_candidates/flarecandi...
1,2011-02-08 21:11:00+00:00,C4.0,/mnt/e/ALEXIS_data/flare_candidates/flarecandi...


In [74]:
# keep track of flare number w/ k


flare_candidate_dir


k = len(WD_already_done)

In [75]:
# THESE_FLARES.working_dir.iloc[0]

In [76]:
def convert_int_class_to_float(flare_class):

    """ 
        make sure flare class is not C1 or M1 but C1.0 or M1.0 etc.
    """
    
    if len(flare_class) == 2:
        flare_letter = flare_class[:1]
        flare_number = np.float(flare_class[1:])
        flare_class = f'{flare_letter}{flare_number}'

    return(flare_class)

def create_working_dir(flare_class, flare_time):
    this_file_time = f'{flare_time:%Y-%m-%dT%H_%M_%S_%f}'[:-4] # cut the milisecond precision to seconds
    name_of_directory = f'flarecandidate_{flare_class}_at_{this_file_time}.working'
    return(name_of_directory)

tw = lambda x: os.path.join(WORKING_DIR, x)

## end of some functions

# workdir variable
WORKING_DIR = dataconfig.DATA_DIR_FLARE_CANDIDATES

In [77]:
WORKING_DIR

'/mnt/e/ALEXIS_data/flare_candidates'

In [78]:

# make working directory paths
THESE_FLARES['working_dir'] = [tw(create_working_dir(this_class, this_time)) for this_class, this_time in zip(THESE_FLARES['merged_class'], THESE_FLARES['merged_datetime'])]
THESE_FLARES.working_dir.iloc[0]

'/mnt/e/ALEXIS_data/flare_candidates/flarecandidate_C4.0_at_2011-02-08T21_11_00_08.working'

In [79]:
DL_DATASETS = []

for flare_candidate in THESE_FLARES.itertuples():

    # data_directory = f'{dataconfig.MAIN_DIR}/image_data'

    the_timestamp = flare_candidate.merged_datetime

    aia_avail_df = query_the_data.aia_availability(input_datetime = the_timestamp, query_time = 20) #query_time must be given in minutes

    # save aia_sql result with filename instead of file_name. Change in order to continue
    aia_avail_df.rename(columns = {'filename':'file_name'}, inplace = True)

    # sxi_avail_df = query_the_data.sxi_availability_sql_db(input_datetime = the_timestamp, query_time = 40) #query_time must be given in minutes

    # #sci qual SXI is data_level == 'BA'
    # sci_qual_sxi_df = sxi_avail_df[sxi_avail_df.data_level == 'BA']

    # sci_qual_sxi_df['url'] = [re.search(r'ftp://satdat.ngdc.noaa.gov/sxi/archive/fits/goes\d{2}/\d{4}/\d{2}/\d{2}/SXI_\d{8}_\d{9}_BA_\d{2}.FTS', this_download_string).group(0) for this_download_string in sci_qual_sxi_df.download_string]
    # [re.search(r'https:\/\/www.ncei.noaa.gov\/data\/goes-solar-xray-imager\/access\/fits\/goes\d{2}\/\d{4}\/\d{2}\/\d{2}\/SXI_\d{8}_\d{9}_[A-B][A-B]_\d{2}.FTS', this_download_string).group(0) for this_download_string in available_data_df.download_string]

    #drop columns to make aia and SXI the same dictionaries

    # PASS_THIS_SXI = sci_qual_sxi_df.drop(['download_string', 'data_level', 'instrument'], axis = 1)

    PASS_THIS_AIA = aia_avail_df.drop(['EXPTIME', 'QUALITY', 'WAVELNTH'], axis = 1)


    # lst_concat_df = pd.concat([PASS_THIS_SXI, PASS_THIS_AIA])

    lst_concat_df = PASS_THIS_AIA


    lst_concat_df['entry_num'] = [k for _ in lst_concat_df.url]

    # lst_concat_df = aia_avail_df

    # k = k + 1

    DL_DATASETS.append(lst_concat_df)

k = k + 1
# all data has been found, concat into dataframe of every single file
THESE_DF = pd.concat(DL_DATASETS)

# drop any duplicate urls

not_duplicated_urls = THESE_DF.drop_duplicates(subset=['url']).reset_index(drop = True)

In [80]:
not_duplicated_urls

,time_stamp,url,file_name,date_time,entry_num
0,1.297198e+09,http://jsoc.stanford.edu/SUM75/D134838226/S000...,aia_lev1_171a_2011_02_08t20_51_01_34z_image_le...,2011-02-08 20:51:01.340000+00:00,0
1,1.297198e+09,http://jsoc.stanford.edu/SUM79/D134838229/S000...,aia_lev1_211a_2011_02_08t20_51_02_07z_image_le...,2011-02-08 20:51:02.070000128+00:00,0
2,1.297198e+09,http://jsoc.stanford.edu/SUM76/D134838234/S000...,aia_lev1_94a_2011_02_08t20_51_03_57z_image_lev...,2011-02-08 20:51:03.569999872+00:00,0
3,1.297198e+09,http://jsoc.stanford.edu/SUM78/D134838237/S000...,aia_lev1_335a_2011_02_08t20_51_05_07z_image_le...,2011-02-08 20:51:05.070000128+00:00,0
4,1.297198e+09,http://jsoc.stanford.edu/SUM78/D134838245/S000...,aia_lev1_193a_2011_02_08t20_51_08_84z_image_le...,2011-02-08 20:51:08.840000+00:00,0
...,...,...,...,...,...
1392,1.297201e+09,http://jsoc.stanford.edu/SUM74/D134841143/S000...,aia_lev1_94a_2011_02_08t21_30_51_57z_image_lev...,2011-02-08 21:30:51.569999872+00:00,0
1393,1.297201e+09,http://jsoc.stanford.edu/SUM80/D134841146/S000...,aia_lev1_335a_2011_02_08t21_30_53_08z_image_le...,2011-02-08 21:30:53.080000+00:00,0
1394,1.297201e+09,http://jsoc.stanford.edu/SUM77/D134841151/S000...,aia_lev1_193a_2011_02_08t21_30_56_84z_image_le...,2011-02-08 21:30:56.840000+00:00,0
1395,1.297201e+09,http://jsoc.stanford.edu/SUM80/D134841153/S000...,aia_lev1_304a_2011_02_08t21_30_57_58z_image_le...,2011-02-08 21:30:57.580000+00:00,0


In [81]:
download_these = not_duplicated_urls.to_dict('records')

download_these

[{'time_stamp': 1297198261.34,
  'url': 'http://jsoc.stanford.edu/SUM75/D134838226/S00000/image_lev1.fits',
  'file_name': 'aia_lev1_171a_2011_02_08t20_51_01_34z_image_lev1.fits',
  'date_time': Timestamp('2011-02-08 20:51:01.340000+0000', tz='UTC'),
  'entry_num': 0},
 {'time_stamp': 1297198262.0700002,
  'url': 'http://jsoc.stanford.edu/SUM79/D134838229/S00000/image_lev1.fits',
  'file_name': 'aia_lev1_211a_2011_02_08t20_51_02_07z_image_lev1.fits',
  'date_time': Timestamp('2011-02-08 20:51:02.070000128+0000', tz='UTC'),
  'entry_num': 0},
 {'time_stamp': 1297198263.57,
  'url': 'http://jsoc.stanford.edu/SUM76/D134838234/S00000/image_lev1.fits',
  'file_name': 'aia_lev1_94a_2011_02_08t20_51_03_57z_image_lev1.fits',
  'date_time': Timestamp('2011-02-08 20:51:03.569999872+0000', tz='UTC'),
  'entry_num': 0},
 {'time_stamp': 1297198265.0700002,
  'url': 'http://jsoc.stanford.edu/SUM78/D134838237/S00000/image_lev1.fits',
  'file_name': 'aia_lev1_335a_2011_02_08t20_51_05_07z_image_lev1.fi

In [82]:
def dl_params(config):
    # for config in download_these[:1]:

    infile = None

    url = config['url']

    hashed_url = hashlib.sha256(url.encode('utf-8')).hexdigest()

    file_syst_sub_dir = hashed_url[:2]

    config['file_syst_sub_dir'] = file_syst_sub_dir

    outfile = f'{dataconfig.DATA_DIR_IMG_RUFFUS_OUTPUT}/{file_syst_sub_dir}/{hashed_url}.downloaded'

    config['ruffus_download_file'] = outfile

    config['hashed_url'] = hashed_url

    return(infile, outfile, config)

In [83]:
config = dl_params(download_these[0])
config

(None,
 '/mnt/e/ALEXIS_data/image_data/ruffus_output_files/45/45e6dd69fd2240b23ce66560539d310658949105f4bf94a379851ad9cdb5afea.downloaded',
 {'time_stamp': 1297198261.34,
  'url': 'http://jsoc.stanford.edu/SUM75/D134838226/S00000/image_lev1.fits',
  'file_name': 'aia_lev1_171a_2011_02_08t20_51_01_34z_image_lev1.fits',
  'date_time': Timestamp('2011-02-08 20:51:01.340000+0000', tz='UTC'),
  'entry_num': 0,
  'file_syst_sub_dir': '45',
  'ruffus_download_file': '/mnt/e/ALEXIS_data/image_data/ruffus_output_files/45/45e6dd69fd2240b23ce66560539d310658949105f4bf94a379851ad9cdb5afea.downloaded',
  'hashed_url': '45e6dd69fd2240b23ce66560539d310658949105f4bf94a379851ad9cdb5afea'})

In [84]:

all_path_info_w_config = download_module_w_new_file_syst.all_paths_config_and_hashed_url(dataconfig.DATA_DIR_IMG_DATA, config[2])

In [85]:
all_path_info_w_config.cached_path_w_sub

'/mnt/e/ALEXIS_data/image_data/cached_directory/45/45e6dd69fd2240b23ce66560539d310658949105f4bf94a379851ad9cdb5afea.cached.pickle'

In [86]:
all_path_info_w_config.fits_path_w_sub


'/mnt/e/ALEXIS_data/image_data/fits_data/45/45e6dd69fd2240b23ce66560539d310658949105f4bf94a379851ad9cdb5afea'

In [87]:
sub_dir = all_path_info_w_config.CONFIG_FILE['file_syst_sub_dir']

ERASE_THESE_FILES = os.path.join(dataconfig.DATA_DIR_IMG_DATA, f'*/{sub_dir}/{all_path_info_w_config.hashed_url}*')

In [88]:
ERASE_THESE_FILES

'/mnt/e/ALEXIS_data/image_data/*/45/45e6dd69fd2240b23ce66560539d310658949105f4bf94a379851ad9cdb5afea*'

In [89]:

output = all_path_info_w_config.CONFIG_FILE['file_name']

# this_file = f'~/jorge-helio/argonne_files/no_space_error_cache/{output}.txt'

this_file = f'{dataconfig.DATA_DIR_IMG_FITS_DATA}/bad_download_cache/{output}'

this_file

'/mnt/e/ALEXIS_data/image_data/fits_data/bad_download_cache/aia_lev1_171a_2011_02_08t20_51_01_34z_image_lev1.fits'